In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from math import sqrt
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import get_latest_prices
from dateutil.relativedelta import relativedelta
import copy
import warnings
warnings.filterwarnings('ignore')

tickers = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'INTC', 'AMD', 'NVDA', 'F', 'TSLA', 'JPM', 'MS', 'VOO']
data_for_portfolio_5y = yf.download(tickers, start='2017-01-01', end='2022-01-01')['Close'].dropna()
data_for_portfolio_5y.tail()

[*********************100%***********************]  12 of 12 completed


,AAPL,AMD,AMZN,F,GOOG,INTC,JPM,MS,MSFT,NVDA,TSLA,VOO
Date,,,,,,,,,,,,
2021-12-27,180.330002,154.360001,169.669495,20.799999,148.063995,51.939999,158.160004,100.400002,342.450012,309.450012,364.646667,438.809998
2021-12-28,179.289993,153.149994,170.660995,20.760000,146.447998,51.759998,158.639999,99.970001,341.250000,303.220001,362.823334,438.480011
2021-12-29,179.380005,148.259995,169.201004,20.559999,146.504501,51.830002,158.559998,98.730003,341.950012,300.010010,362.063324,439.010010
2021-12-30,178.199997,145.149994,168.644501,20.469999,146.002502,51.740002,158.479996,98.800003,339.320007,295.859985,356.779999,437.769989
2021-12-31,177.570007,143.899994,166.716995,20.770000,144.679504,51.500000,158.350006,98.160004,336.320007,294.109985,352.260010,436.570007


In [2]:
# Create initial portfolio
mu = mean_historical_return(data_for_portfolio_5y)
S = CovarianceShrinkage(data_for_portfolio_5y).ledoit_wolf()
ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
weights = ef.max_sharpe()
initial_weights = ef.clean_weights()
print(initial_weights, '\n')

# Initial buy stocks
balance_usd = 100_000
balance_remain = balance_usd
stocks_initial = {}
latest_prices_data_dec_2021 = get_latest_prices(data_for_portfolio_5y)
for ticker in initial_weights:
    to_spend_usd = balance_usd * initial_weights[ticker]
    one_stock_price = latest_prices_data_dec_2021[ticker]
    stocks_number = to_spend_usd // one_stock_price
    stocks_initial[ticker] = stocks_number
    balance_remain = balance_remain - (stocks_number * one_stock_price)
print('Portfolio:', stocks_initial)
print('Remaining balance: USD', balance_remain)

OrderedDict([('AAPL', 0.25899), ('AMD', 0.10384), ('AMZN', 0.0), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.25217), ('NVDA', 0.09699), ('TSLA', 0.288), ('VOO', 0.0)]) 

Portfolio: {'AAPL': 145.0, 'AMD': 72.0, 'AMZN': 0.0, 'F': 0.0, 'GOOG': 0.0, 'INTC': 0.0, 'JPM': 0.0, 'MS': 0.0, 'MSFT': 74.0, 'NVDA': 32.0, 'TSLA': 81.0, 'VOO': 0.0}
Remaining balance: USD 1059.2885131835938


In [3]:
# Calculate MACD, RSI, and S&R signals.
data_2022 = yf.download(tickers, start='2021-12-14', end='2023-01-01')['Close'].dropna()

short_ma = 5                                       # 5 минулих днів для "короткого" МА 
long_ma = 12                                       # 12 минулих днів для "довгого" МА 
rsi_period = 14                                    # RSI рахується за 14 минулих днів 
rsi_oversold = 30                                  # Параметри для індикатора RSI 
rsi_overbought = 70 
sr_sell = 0.7                                      # Параметри для індикатора S&R
sr_buy = 0.3 
start = max(long_ma, rsi_period)

for ticker in data_2022.columns:
    data_2022['MA_' + ticker + str(short_ma)] = data_2022[ticker].rolling(short_ma).mean()
    data_2022['MA_' + ticker + str(long_ma)] = data_2022[ticker].rolling(long_ma).mean() 
    data_2022['return_' + ticker] = data_2022[ticker].pct_change() 
    data_2022['Up_' + ticker] = np.maximum(data_2022[ticker].diff(),0) 
    data_2022['Down_' + ticker] = np.maximum(-data_2022[ticker].diff(),0) 
    data_2022['RS_' + ticker] = data_2022['Up_' + ticker].rolling(rsi_period).mean()/data_2022['Down_' + ticker].rolling(rsi_period).mean() 
    data_2022['RSI_' + ticker] = 100 - 100/(1 + data_2022['RS_' + ticker]) 
    data_2022['S&R_' + ticker] = (data_2022[ticker]/(10**np.floor(np.log10(data_2022[ticker]))))%1 
    data_2022['MACD_signal_' + ticker] = 2*(data_2022['MA_' + ticker + str(short_ma)] > data_2022['MA_' + ticker + str(long_ma)]) -1 
    data_2022['RSI_signal_' + ticker] = 1*(data_2022['RSI_' + ticker] < rsi_oversold) -1*(data_2022['RSI_' + ticker] > rsi_overbought) 
    data_2022['S&R_signal_' + ticker] = 1*(data_2022['S&R_' + ticker] < sr_buy) -1*(data_2022['S&R_' + ticker] > sr_sell) 

data = data_2022.drop(data_2022.index[range(14)])
#print(data.to_string())
data.head(3)

[*********************100%***********************]  12 of 12 completed


,AAPL,AMD,AMZN,F,GOOG,INTC,JPM,MS,MSFT,NVDA,...,MA_VOO12,return_VOO,Up_VOO,Down_VOO,RS_VOO,RSI_VOO,S&R_VOO,MACD_signal_VOO,RSI_signal_VOO,S&R_signal_VOO
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,182.009995,150.240005,170.404495,21.770000,145.074493,53.209999,161.699997,100.190002,334.750000,301.209991,...,432.603335,0.006139,2.679993,0.000000,1.549355,60.774396,0.3925,1,0,0
2022-01-04,179.699997,144.419998,167.522003,24.309999,144.416504,53.139999,167.830002,104.260002,329.010010,292.899994,...,433.455836,-0.000410,0.000000,0.179993,1.816353,64.493090,0.3907,1,0,0
2022-01-05,174.919998,136.149994,164.356995,23.660000,137.653503,53.869999,163.779999,101.680000,316.380005,276.040009,...,433.978335,-0.019154,0.000000,8.410004,0.914027,47.754141,0.3066,1,0,0


In [4]:
data.tail(3)

,AAPL,AMD,AMZN,F,GOOG,INTC,JPM,MS,MSFT,NVDA,...,MA_VOO12,return_VOO,Up_VOO,Down_VOO,RS_VOO,RSI_VOO,S&R_VOO,MACD_signal_VOO,RSI_signal_VOO,S&R_signal_VOO
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-28,126.040001,62.570000,81.82,10.95,86.459999,25.540001,132.460007,84.459999,234.529999,140.360001,...,355.751668,-0.012269,0.000000,4.299988,0.541579,35.131445,0.4617,-1,0,0
2022-12-29,129.610001,64.820000,84.18,11.54,88.949997,26.209999,133.220001,85.239998,241.009995,146.029999,...,354.554169,0.017737,6.139984,0.000000,0.641064,39.063936,0.5231,-1,0,0
2022-12-30,129.929993,64.769997,84.00,11.63,88.730003,26.430000,134.100006,85.019997,239.820007,146.139999,...,353.050001,-0.002753,0.000000,0.970001,0.675590,40.319512,0.5134,-1,0,0


In [5]:
def check_tickers_for_sell(tickers_sell, now, current_weights):
    ticker_percent_sell = {}
    end_date = now.strftime('%Y-%m-%d')
    start_date = (now - relativedelta(years=5)).strftime('%Y-%m-%d')
    # Get data for all tickers for last 5 years
    data_for_tickers = yf.download(tickers, start_date, end_date, progress=False)['Close'].dropna()
    
    # Generate portfolio on this data
    mu = mean_historical_return(data_for_tickers)
    S = CovarianceShrinkage(data_for_tickers).ledoit_wolf()
    ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
    weights = ef.max_sharpe()
    new_weights = ef.clean_weights()
    
    for ticker in tickers_sell:
        if new_weights[ticker] == 0:
            ticker_percent_sell[ticker] = 100
        elif new_weights[ticker] < current_weights[ticker]:
            delta = current_weights[ticker] - new_weights[ticker]
            sell_percent = (delta * 100) // current_weights[ticker]
            ticker_percent_sell[ticker] = sell_percent
    return ticker_percent_sell

In [6]:
def select_ticker_for_sell(data, date, filtered_sell_tickers_with_percents, stocks_current):
    sell_ticker = ''
    sell_profit = 0
    stocks_to_sell = 0
    for ticker in filtered_sell_tickers_with_percents:
        no_of_stocks = stocks_current[ticker]
        #print(ticker, 'no_of_stocks', no_of_stocks)
        cur_price = data[ticker][date]
        #print(ticker, 'cur_price', cur_price)
        sell_percent = (filtered_sell_tickers_with_percents[ticker] / 100)
        #print(ticker, 'sell_percent', sell_percent)
        potentail_profit = no_of_stocks * cur_price * sell_percent
        #print(ticker, 'potentail_profit', potentail_profit)
        if potentail_profit > sell_profit:
            sell_profit = potentail_profit
            sell_ticker = ticker
            stocks_to_sell = no_of_stocks * sell_percent
    return [sell_ticker, sell_profit, stocks_to_sell]

In [7]:
def select_ticker_for_buy(tickers_buy, now):
    if len(tickers_buy) == 1:
        return tickers_buy[0]
    end_date = now.strftime('%Y-%m-%d')
    start_date = (now - relativedelta(years=5)).strftime('%Y-%m-%d')
    # Get data for all tickers for last 5 years period
    data_for_tickers_last_year = yf.download(tickers_buy, start_date, end_date, progress=False)['Close'].dropna()
    ticker_percent_delta = {}
    for t in tickers_buy:
        first_price = data_for_tickers_last_year[t][0]
        last_price = data_for_tickers_last_year[t][len(data_for_tickers_last_year[t]) - 1]
        delta = last_price - first_price
        delta_abs = abs(delta)
        percent = (delta_abs * 100) / first_price
        if delta > 0:
            ticker_percent_delta[t] = percent
        else:
            ticker_percent_delta[t] = -1 * percent
    sorted_by_delta = dict(sorted(ticker_percent_delta.items(), key=lambda x:x[1], reverse=True))
    return list(sorted_by_delta.keys())[0]

In [8]:
print('Start weights:', initial_weights)
clean_result = balance_remain
for stock in initial_weights:
    clean_result = clean_result + (stocks_initial[stock] * data[stock][len(data[stock]) - 1])
print('Without any changes balance on the end of 2022 would be', clean_result)
print('------------------------------------')

Start weights: OrderedDict([('AAPL', 0.25899), ('AMD', 0.10384), ('AMZN', 0.0), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.25217), ('NVDA', 0.09699), ('TSLA', 0.288), ('VOO', 0.0)])
Without any changes balance on the end of 2022 would be 56963.31775665283
------------------------------------


In [9]:
strategy = 'MACD_signal_'                 # MACD_signal_        or        RSI_signal_        or        S&R_signal_
strategy_weights = copy.deepcopy(initial_weights)
stocks_strategy = copy.deepcopy(stocks_initial)
balance_strategy = balance_remain

balance_buy_and_hold_dict = {}
balance_strategy_dict = {}

for d in data.index: 
    if d.weekday() == 4:                   # every Friday
        tickers_sell = []
        tickers_buy = []
        for t in strategy_weights:
            if data[strategy + t][d] == -1:
                tickers_sell.append(t)
            if data[strategy + t][d] == 1:
                tickers_buy.append(t)
        print(d, 'Sell candidates:', tickers_sell, 'Buy candidates:', tickers_buy)
        if len(tickers_sell) > 0 and len(tickers_buy) > 0:
            filtered_sell_tickers_with_percents = check_tickers_for_sell(tickers_sell, d, strategy_weights)
            print('Filtered sell candidates:', filtered_sell_tickers_with_percents)
            if len(filtered_sell_tickers_with_percents) > 0:
                ticker_for_sell_data = select_ticker_for_sell(data, d, filtered_sell_tickers_with_percents, stocks_strategy)
                will_receive_usd = ticker_for_sell_data[1]
                # If this ticker is present in current portfolio
                if will_receive_usd > 0:
                    ticker_for_sell = ticker_for_sell_data[0]
                    stocks_to_sell = ticker_for_sell_data[2]
                    sell_percents = filtered_sell_tickers_with_percents[ticker_for_sell]
                    ticker_for_buy = select_ticker_for_buy(tickers_buy, d)
                    # If we can buy other ticker on that money
                    if (will_receive_usd + balance_strategy) > data[ticker_for_buy][d]:
                        print('Will sell',ticker_for_sell,'and get',will_receive_usd,'$ after selling (',sell_percents,'% )')
                        print('Will buy', ticker_for_buy)
                        # Sell percent of ticker_for_sell and add that money to balance
                        stocks_strategy[ticker_for_sell] = stocks_strategy[ticker_for_sell] - stocks_to_sell
                        balance_strategy = balance_strategy + will_receive_usd
                        # Buy ticker_for_buy on all balance
                        no_of_buy_stocks = balance_strategy // data[ticker_for_buy][d]
                        stocks_strategy[ticker_for_buy] = stocks_strategy[ticker_for_buy] + no_of_buy_stocks
                        balance_strategy = balance_strategy - (data[ticker_for_buy][d] * no_of_buy_stocks)
                        # Recalculate new weights for all portfolio
                        # calculate all prices for now + balance - потом высчитать сколько в процентах сейчас каждый ФИ
                        cur_balance = balance_strategy
                        stock_usd = {}
                        for stock in stocks_strategy:
                            stock_usd[stock] = stocks_strategy[stock] * data[stock][d]
                            cur_balance = cur_balance + stock_usd[stock]
                        for stock in stocks_strategy:
                            strategy_weights[stock] = stock_usd[stock] / cur_balance
                        print('Current weights are', strategy_weights)
                        print('Current balance =', cur_balance)
                    else:
                        print("No changes because sell profit is not enough for buying at least one stock.")
                else:
                    print("No changes because all sell candidates are not present in current portfolio.")
            else:
                print("No changes because all sell candidates failed check.")
        else:
            print("No changes because no sell candidates or no buy candidates.")
        print('------------------------------------')
    # Calculate current balance for stocks_initial
    today_balance_buy_and_hold = balance_remain
    for stock in stocks_initial:
        today_balance_buy_and_hold = today_balance_buy_and_hold + (data[stock][d] * stocks_initial[stock])
    balance_buy_and_hold_dict[d] = today_balance_buy_and_hold
    # Calculate current balance for stocks_strategy
    today_balance_strategy = balance_strategy
    for stock in stocks_strategy:
        today_balance_strategy = today_balance_strategy + (data[stock][d] * stocks_strategy[stock])
    balance_strategy_dict[d] = today_balance_strategy

print('\nWith strategy final weights are:', strategy_weights)        
clean_result = balance_strategy
for stock in strategy_weights:
    clean_result = clean_result + (stocks_strategy[stock] * data[stock][len(data[stock]) - 1])
print('\nWith strategy balance on the end of 2022 would be', clean_result)

2022-01-07 00:00:00 Sell candidates: ['AAPL', 'AMD', 'AMZN', 'GOOG', 'MSFT', 'NVDA', 'VOO'] Buy candidates: ['F', 'INTC', 'JPM', 'MS', 'TSLA']
Filtered sell candidates: {'AMZN': 100, 'GOOG': 100, 'MSFT': 18.0, 'VOO': 100}
Will sell MSFT and get 4183.012913818359 $ after selling ( 18.0 % )
Will buy TSLA
Current weights are OrderedDict([('AAPL', 0.26219554445500065), ('AMD', 0.0998174010451634), ('AMZN', 0.0), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.2001383812830141), ('NVDA', 0.09157322351724945), ('TSLA', 0.34514639855419005), ('VOO', 0.0)])
Current balance = 95213.8595123291
------------------------------------
2022-01-14 00:00:00 Sell candidates: ['AAPL', 'AMD', 'AMZN', 'GOOG', 'MSFT', 'NVDA', 'TSLA', 'VOO'] Buy candidates: ['F', 'INTC', 'JPM', 'MS']
Filtered sell candidates: {'AMZN': 100, 'GOOG': 100, 'MSFT': 5.0, 'NVDA': 21.0, 'TSLA': 16.0, 'VOO': 100}
Will sell TSLA and get 5374.003125 $ after selling ( 16.0 % )
Will buy MS
Current weights a

Filtered sell candidates: {'AAPL': 66.0, 'AMZN': 100, 'F': 100, 'GOOG': 100, 'INTC': 100, 'JPM': 100, 'MS': 100, 'NVDA': 100, 'VOO': 100}
Will sell MS and get 18255.100631713867 $ after selling ( 100 % )
Will buy AMD
Current weights are OrderedDict([('AAPL', 0.14816488300068237), ('AMD', 0.28366365824963835), ('AMZN', 0.0), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.21426068622820768), ('NVDA', 0.1190317850039368), ('TSLA', 0.23362323660589338), ('VOO', 0.0)])
Current balance = 71526.61051189086
------------------------------------
2022-05-27 00:00:00 Sell candidates: ['AMZN', 'F', 'GOOG', 'INTC', 'TSLA'] Buy candidates: ['AAPL', 'AMD', 'JPM', 'MS', 'MSFT', 'NVDA', 'VOO']
Filtered sell candidates: {'AMZN': 100, 'F': 100, 'GOOG': 100, 'INTC': 100}
No changes because all sell candidates are not present in current portfolio.
------------------------------------
2022-06-03 00:00:00 Sell candidates: [] Buy candidates: ['AAPL', 'AMD', 'AMZN', 'F', 'GOOG',

Filtered sell candidates: {'AMD': 53.0, 'TSLA': 30.0}
Will sell TSLA and get 7577.486454542297 $ after selling ( 30.0 % )
Will buy MSFT
Current weights are OrderedDict([('AAPL', 0.16672969750711714), ('AMD', 0.11996813546586671), ('AMZN', 0.0), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.17928304052638927), ('MSFT', 0.2585797095449863), ('NVDA', 0.0), ('TSLA', 0.2732006471205743), ('VOO', 0.0)])
Current balance = 64717.27616172929
------------------------------------
2022-10-14 00:00:00 Sell candidates: ['AAPL', 'AMD', 'AMZN', 'F', 'GOOG', 'INTC', 'JPM', 'MS', 'MSFT', 'NVDA', 'TSLA', 'VOO'] Buy candidates: []
No changes because no sell candidates or no buy candidates.
------------------------------------
2022-10-21 00:00:00 Sell candidates: ['AMD', 'MS', 'TSLA'] Buy candidates: ['AAPL', 'AMZN', 'F', 'GOOG', 'INTC', 'JPM', 'MSFT', 'NVDA', 'VOO']
Filtered sell candidates: {'AMD': 100, 'MS': 100}
Will sell MS and get 11645.34017944336 $ after selling ( 100 % )
Will bu

In [10]:
high_buy_and_hold = 0
low_buy_and_hold = 1_000_000_000
high_strategy = 0
low_strategy = 1_000_000_000
biggest_delta = 0
biggest_delta_day = ''
final_delta_usd = 0
final_delta_percent = 0

for d in balance_buy_and_hold_dict:
    if balance_buy_and_hold_dict[d] > high_buy_and_hold:
        high_buy_and_hold = balance_buy_and_hold_dict[d]
    if balance_buy_and_hold_dict[d] < low_buy_and_hold:
        low_buy_and_hold = balance_buy_and_hold_dict[d]
    if balance_strategy_dict[d] > high_strategy:
        high_strategy = balance_strategy_dict[d]
    if balance_strategy_dict[d] < low_strategy:
        low_strategy = balance_strategy_dict[d]
    delta = balance_strategy_dict[d] - balance_buy_and_hold_dict[d]
    delta_abs = abs(delta)
    percent = delta_abs / balance_buy_and_hold_dict[d]
    percent_change = percent
    if delta < 0:
        percent_change = -1 * percent_change
    if percent > abs(biggest_delta):
        biggest_delta = percent_change
        biggest_delta_day = d
    final_delta_usd = balance_strategy_dict[d] - balance_buy_and_hold_dict[d]
    final_delta_percent = percent_change
    print(d.strftime('%Y-%m-%d'), '\tBuy and hold balance:\t', "{:.2f}".format(balance_buy_and_hold_dict[d]), 
          '\tStrategy balance:\t', "{:.2f}".format(balance_strategy_dict[d]), 
          '\tDelta:\t', "{:.6f}".format(percent_change))

print('\nhigh_buy_and_hold', high_buy_and_hold)
print('low_buy_and_hold', low_buy_and_hold)
print('\nhigh_strategy', high_strategy)
print('low_strategy', low_strategy)
print('\nbiggest_delta', biggest_delta)
print('biggest_delta_day', biggest_delta_day)
print('\nfinal_delta_usd', final_delta_usd)
print('final_delta_percent', final_delta_percent)

2022-01-03 	Buy and hold balance:	 105072.30 	Strategy balance:	 105072.30 	Delta:	 0.000000
2022-01-04 	Buy and hold balance:	 102272.50 	Strategy balance:	 102272.50 	Delta:	 0.000000
2022-01-05 	Buy and hold balance:	 97850.13 	Strategy balance:	 97850.13 	Delta:	 0.000000
2022-01-06 	Buy and hold balance:	 96798.83 	Strategy balance:	 96798.83 	Delta:	 0.000000
2022-01-07 	Buy and hold balance:	 95213.86 	Strategy balance:	 95213.86 	Delta:	 0.000000
2022-01-10 	Buy and hold balance:	 96124.06 	Strategy balance:	 96276.79 	Delta:	 0.001589
2022-01-11 	Buy and hold balance:	 97280.97 	Strategy balance:	 97455.65 	Delta:	 0.001796
2022-01-12 	Buy and hold balance:	 98788.58 	Strategy balance:	 99128.53 	Delta:	 0.003441
2022-01-13 	Buy and hold balance:	 94495.44 	Strategy balance:	 94641.52 	Delta:	 0.001546
2022-01-14 	Buy and hold balance:	 95925.51 	Strategy balance:	 96089.91 	Delta:	 0.001714
2022-01-18 	Buy and hold balance:	 93688.08 	Strategy balance:	 93687.48 	Delta:	 -0.0

2022-12-06 	Buy and hold balance:	 64660.82 	Strategy balance:	 63847.11 	Delta:	 -0.012584
2022-12-07 	Buy and hold balance:	 63884.69 	Strategy balance:	 63007.45 	Delta:	 -0.013732
2022-12-08 	Buy and hold balance:	 64667.70 	Strategy balance:	 63335.76 	Delta:	 -0.020597
2022-12-09 	Buy and hold balance:	 64715.42 	Strategy balance:	 63020.07 	Delta:	 -0.026197
2022-12-12 	Buy and hold balance:	 64988.94 	Strategy balance:	 63204.39 	Delta:	 -0.027459
2022-12-13 	Buy and hold balance:	 65143.31 	Strategy balance:	 63243.25 	Delta:	 -0.029167
2022-12-14 	Buy and hold balance:	 64178.46 	Strategy balance:	 61604.71 	Delta:	 -0.040103
2022-12-15 	Buy and hold balance:	 62264.60 	Strategy balance:	 59716.58 	Delta:	 -0.040922
2022-12-16 	Buy and hold balance:	 60851.17 	Strategy balance:	 58310.81 	Delta:	 -0.041747
2022-12-19 	Buy and hold balance:	 60037.47 	Strategy balance:	 57592.61 	Delta:	 -0.040722
2022-12-20 	Buy and hold balance:	 59128.59 	Strategy balance:	 56576.92 	Delta: